In [ ]:
import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

from dateutil.parser import parse
import multiprocessing
import pathlib
from scipy import signal
from scipy.io import wavfile
import re
import string

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
import tensorflow as tf
print(tf.__version__)
#import keras
import tf_keras as tfkr
print(tfkr.__version__)
from tensorflow.python.client import device_lib

In [ ]:
tf.config.list_physical_devices('GPU')

In [3]:
datapath = "/kaggle/input/covid-tweets-dataset"
tweet_data_1 = pd.read_csv(os.path.join(datapath, "processed_tweets_1_updated.csv"))
tweet_data_1_backup = tweet_data_1.copy()
tweet_data_2 = pd.read_csv(os.path.join(datapath, "processed_tweets_2_updated.csv"))
tweet_data_2_backup = tweet_data_2.copy()
tweet_data_3 = pd.read_csv(os.path.join(datapath, "processed_tweets_3_updated.csv"))
tweet_data_3_backup = tweet_data_3.copy()

In [4]:
tweet_data_1.head(5)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,original_text,favorite_count,retweet_count,original_author,hashtags,user_mentions,place,clean_tweet,compound,neg,neu,pos,sentiment,text_len,processed_text,is_retweet,tokens_count,spell_error_count,spell_error_words,unknown_words
0,RT @GlblCtzn: .@priyankachopra is calling on l...,0.0,31.0,RJIshak,NaN,"GlblCtzn, priyankachopra",Jakarta Capital Region,call leader help protect refuge covid19 provid...,0.8176,0.0,0.452,0.548,pos,139,call leader help protect refugee covid provide...,True,10.0,0.0,NaN,NaN
1,RT @OGSG_Official: OGUN STATE SUPPORT FOR CBN-...,0.0,61.0,makinwaoluwole,NaN,OGSG_Official,Nigeria,ogun state support cbn nirsal covid19 target c...,0.6486,0.0,0.602,0.398,pos,116,state support covid target credit facility,True,6.0,3.0,"ogun, cbn-nirsal, tcf",NaN
2,RT @AdvoBarryRoux: These 5 police officials ba...,0.0,1.0,TembeAmu,NaN,AdvoBarryRoux,NaN,polic offici base namahadi polic station busi ...,0.2732,0.0,0.851,0.149,pos,139,police official base police station busy drink...,True,12.0,1.0,namahadi,NaN
3,RT @MobilePunch: COVID-19: Oyo discharges two ...,0.0,0.0,ilyasrabiu,NaN,MobilePunch,"Lagos, Nigeria",covid19 oyo discharg two patient,0.0000,0.0,1.000,0.000,neu,78,covid discharge two patient,True,4.0,1.0,oyo,NaN
4,My Condolences to the Family of those who did ...,0.0,13869.0,bucketeconomist,covid_19,NaN,NaN,condol famili surviv,0.0000,0.0,1.000,0.000,neu,68,condolence family survive,False,3.0,0.0,NaN,NaN


In [ ]:
tweet_data_1.info()

In [5]:
#drop_columns = ['id', 'source', 'lang']
drop_columns = ['place', 'original_author']
#dropna_columns = ['original_text', 'clean_tweet', 'lang']
dropna_columns = ['original_text', 'clean_tweet', 'processed_text']
tweet_data_1.dropna(subset=dropna_columns, how='any', axis=0, inplace=True)
tweet_data_1.drop(columns=drop_columns, inplace=True)
tweet_data_2.dropna(subset=dropna_columns, how='any', axis=0, inplace=True)
tweet_data_2.drop(columns=drop_columns, inplace=True)
tweet_data_3.dropna(subset=dropna_columns, how='any', axis=0, inplace=True)
tweet_data_3.drop(columns=drop_columns, inplace=True)

# Concatenate the three dataframes into one
tweet_data = pd.concat([tweet_data_1, tweet_data_2, tweet_data_3], ignore_index=True)

# Display the shape of the combined dataframe
tweet_data.shape

(409450, 18)

In [6]:
count_vectorizer = CountVectorizer(ngram_range=(1,1))
all_tweets = tweet_data['processed_text']
#all_tweets = tweet_data['processed_text']
all_tweets_bow = count_vectorizer.fit_transform(all_tweets)
print(all_tweets_bow.shape)

(409450, 32841)


In [7]:
#X = tweet_data['clean_tweet']
X = tweet_data['processed_text']
y = tweet_data['sentiment']
# perform one-hot conversion to y
y = pd.get_dummies(y).astype(float)
# prompt: convert y values from true/false to 1/0

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (327560,)
Shape of X_test: (81890,)
Shape of y_train: (327560, 3)
Shape of y_test: (81890, 3)


In [8]:
from tf_keras import layers
VOCAB_SIZE = 2000
OUTPUT_SEQUENCE_LENGTH = 50
BATCH_SIZE = 256

X_list = X.to_numpy()

#tweets_dataset_all = tf.data.Dataset.from_tensor_slices(X)
#tweets_dataset_all = tweets_dataset_all.batch(BATCH_SIZE)
text_vectorize_layer = layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    split='whitespace',
    output_sequence_length=OUTPUT_SEQUENCE_LENGTH,
)
text_vectorize_layer.adapt(X_list)
vocabulary_all = text_vectorize_layer.get_vocabulary()
print(f"Vocabulary size: {len(vocabulary_all)}")

#X_train_ds = tf.data.Dataset.from_tensor_slices(X_train)
#X_train_ds = X_train_ds.batch(BATCH_SIZE)
#X_test_ds = tf.data.Dataset.from_tensor_slices(X_test)
#X_test_ds = X_test_ds.batch(BATCH_SIZE)
X_train_ds = X_train.to_numpy()
X_test_ds = X_test.to_numpy()

I0000 00:00:1749644867.832437   10192 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1749644867.833226   10192 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Vocabulary size: 2000


In [9]:
print(vocabulary_all[0:30])

['', '[UNK]', 'covid', 'case', 'new', 'vaccine', 'people', 'test', 'amp', 'say', 'death', 'get', 'pandemic', 'health', 'report', 'u', 'one', 'today', 'trump', 'day', 'positive', 'state', 'may', 'take', 'help', 'go', 'time', 'need', 'coronavirus', 'make']


In [10]:
# short test to vectorization layer
string_input = tfkr.Input(shape=(1,), dtype=tf.string)
x = text_vectorize_layer(string_input)
model = tfkr.Model(string_input, x)

test_tweet_vectorized = model.predict(X_test_ds, batch_size=BATCH_SIZE)
print(f"VECTORIZED TEXT: {test_tweet_vectorized[2]}")
reconstructed_text = " ".join([vocabulary_all[word_index] for word_index in test_tweet_vectorized[2]])
print(f"RECONSTRUCTED TEXT: {reconstructed_text}")

print(f"ORIGINAL TEXT: {X_test_ds[2]}")

320/320 [==============================] - 1s 2ms/step
VECTORIZED TEXT: [ 337  591    1    1  788  944 1994  493    1 1947    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]
RECONSTRUCTED TEXT: police officer [UNK] [UNK] arrest driver vehicle drive [UNK] maryland                                        
ORIGINAL TEXT: police officer routine patrol arrest driver vehicle drive traffic maryland


## Simple Embedding Model ##

In [14]:
EMBED_OUT_DIM = 2
EMBED_IN_DIM = VOCAB_SIZE
DENSE_HIDDEN_DIM = 16
tweet_input = tfkr.Input(shape=(1,), dtype=tf.string)
x = text_vectorize_layer(tweet_input)
x = layers.Embedding(input_dim=EMBED_IN_DIM, output_dim=EMBED_OUT_DIM, mask_zero=False)(x)
x = layers.Flatten()(x)
#x = layers.Reshape((-1,))(x)
x = layers.Dense(DENSE_HIDDEN_DIM, activation='relu')(x)
x = layers.Dense(3, activation='sigmoid')(x)
model_embed = tfkr.Model(tweet_input, x)
model_embed.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_embed.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 50)                0         
 ctorization)                                                    
                                                                 
 embedding_1 (Embedding)     (None, 50, 2)             4000      
                                                                 
 flatten_1 (Flatten)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 16)                1616      
                                                                 
 dense_3 (Dense)             (None, 3)                 51        
                                                           

In [15]:
with tf.device('/device:GPU:0'):
    history_embed = model_embed.fit(X_train_ds, y_train, epochs=20, validation_split=0.2, batch_size=BATCH_SIZE, shuffle=True)

Epoch 1/20
512/512 [==============================] - 15s 28ms/step - loss: 0.7886 - accuracy: 0.6529 - val_loss: 0.4199 - val_accuracy: 0.8823
Epoch 2/20
512/512 [==============================] - 4s 8ms/step - loss: 0.3586 - accuracy: 0.8964 - val_loss: 0.3412 - val_accuracy: 0.9011
Epoch 3/20
512/512 [==============================] - 3s 6ms/step - loss: 0.3325 - accuracy: 0.9015 - val_loss: 0.3354 - val_accuracy: 0.9020
Epoch 4/20
512/512 [==============================] - 3s 7ms/step - loss: 0.3279 - accuracy: 0.9025 - val_loss: 0.3337 - val_accuracy: 0.9026
Epoch 5/20
512/512 [==============================] - 3s 6ms/step - loss: 0.3255 - accuracy: 0.9030 - val_loss: 0.3321 - val_accuracy: 0.9034
Epoch 6/20
512/512 [==============================] - 3s 5ms/step - loss: 0.3238 - accuracy: 0.9030 - val_loss: 0.3303 - val_accuracy: 0.9031
Epoch 7/20
512/512 [==============================] - 3s 5ms/step - loss: 0.3223 - accuracy: 0.9031 - val_loss: 0.3295 - val_accuracy: 0.9031
Epoc

In [31]:
y_pred = model_embed.predict(X_test_ds, batch_size=BATCH_SIZE)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test.to_numpy(), axis=1)
# prompt: calculate the accuracy for y_pred
accuracy = np.mean(y_pred == y_true)
print(f"Accuracy: {accuracy}")

160/160 [==============================] - 1s 3ms/step
Accuracy: 0.9035779704481621


In [ ]:
embed_layer = model_embed.layers[2]
embed_layer.input

In [ ]:
embed_layer.output

In [ ]:
test_embed_input = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,998,999,1000,1002])
test_embed_input = test_embed_input.reshape((1, -1))
test_embed_input = tf.convert_to_tensor(test_embed_input, dtype=tf.int32)
test_embed_output = embed_layer(test_embed_input)
print(f"Test embed output shape: {test_embed_output.shape}")

In [ ]:
test_embed_output

## Vanila RNN Model ##

In [ ]:
EMBED_OUT_DIM = 2
EMBED_IN_DIM = VOCAB_SIZE
RNN_HIDDEN_DIM = 16
tweet_input = tfkr.Input(shape=(1,), dtype=tf.string)
x = text_vectorize_layer(tweet_input)
x = layers.Embedding(input_dim=EMBED_IN_DIM, output_dim=EMBED_OUT_DIM, mask_zero=False)(x)
x = layers.SimpleRNN(RNN_HIDDEN_DIM, kernel_initializer=tfkr.initializers.RandomNormal(stddev=0.001),  
                     recurrent_initializer=tfkr.initializers.Identity(gain=1.0), activation='tanh')(x)
x = layers.Dense(3, activation='sigmoid')(x)
model_rnn = tfkr.Model(tweet_input, x)
model_rnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_rnn.summary()

In [ ]:
with tf.device('device:GPU:0'):
    history_rnn = model_rnn.fit(X_train_ds, y_train, epochs=20, validation_split=0.2, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
y_pred = model_rnn.predict(X_test_ds)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test.to_numpy(), axis=1)
# prompt: calculate the accuracy for y_pred
accuracy = np.mean(y_pred == y_true)
print(f"Accuracy: {accuracy}")

## LSTM Model ##

In [18]:
EMBED_OUT_DIM = 2
EMBED_IN_DIM = VOCAB_SIZE
RNN_HIDDEN_DIM = 16
tweet_input = tfkr.Input(shape=(1,), dtype=tf.string)
x = text_vectorize_layer(tweet_input)
x = layers.Embedding(input_dim=EMBED_IN_DIM, output_dim=EMBED_OUT_DIM, mask_zero=False)(x)
x = layers.LSTM(units=RNN_HIDDEN_DIM)(x)
x = layers.Dense(3, activation='sigmoid')(x)
model_lstm = tfkr.Model(tweet_input, x)
model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 50)                0         
 ctorization)                                                    
                                                                 
 embedding_3 (Embedding)     (None, 50, 2)             4000      
                                                                 
 lstm (LSTM)                 (None, 16)                1216      
                                                                 
 dense_5 (Dense)             (None, 3)                 51        
                                                                 
Total params: 5267 (20.57 KB)
Trainable params: 5267 (20.57 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________

In [19]:
with tf.device('device:GPU:0'):
    history_lstm = model_lstm.fit(X_train_ds, y_train, epochs=20, validation_split=0.2, batch_size=BATCH_SIZE, shuffle=True)

Epoch 1/20
512/512 [==============================] - 16s 25ms/step - loss: 1.0518 - accuracy: 0.4430 - val_loss: 0.7470 - val_accuracy: 0.6343
Epoch 2/20
512/512 [==============================] - 5s 10ms/step - loss: 0.6113 - accuracy: 0.7063 - val_loss: 0.4931 - val_accuracy: 0.8365
Epoch 3/20
512/512 [==============================] - 5s 9ms/step - loss: 0.3860 - accuracy: 0.8800 - val_loss: 0.3472 - val_accuracy: 0.8967
Epoch 4/20
512/512 [==============================] - 4s 9ms/step - loss: 0.3364 - accuracy: 0.8990 - val_loss: 0.3420 - val_accuracy: 0.8998
Epoch 5/20
512/512 [==============================] - 4s 9ms/step - loss: 0.3299 - accuracy: 0.9010 - val_loss: 0.3380 - val_accuracy: 0.8989
Epoch 6/20
512/512 [==============================] - 4s 8ms/step - loss: 0.3258 - accuracy: 0.9018 - val_loss: 0.3325 - val_accuracy: 0.9006
Epoch 7/20
512/512 [==============================] - 4s 8ms/step - loss: 0.3239 - accuracy: 0.9026 - val_loss: 0.3312 - val_accuracy: 0.9008
Epo

In [ ]:
y_pred = model_lstm.predict(X_test_ds)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test.to_numpy(), axis=1)
# prompt: calculate the accuracy for y_pred
accuracy = np.mean(y_pred == y_true)
print(f"Accuracy: {accuracy}")

In [ ]:
# save the RNN model
#model_rnn.save(os.path.join(datapath, "tweet_sentiment_rnn_model"), save_format='tf')

## CNN Model ##

In [16]:
EMBED_OUT_DIM = 2
EMBED_IN_DIM = VOCAB_SIZE
CNN_1_SIZE = 5
CNN_1_FILTERS = 24
CNN_2_SIZE = 5
CNN_2_FILTERS = 12


tweet_input = tfkr.Input(shape=(1,), dtype=tf.string)
x = text_vectorize_layer(tweet_input)
x = layers.Embedding(input_dim=EMBED_IN_DIM, output_dim=EMBED_OUT_DIM, mask_zero=False)(x)
x = layers.Conv1D(filters=CNN_1_FILTERS, kernel_size=CNN_1_SIZE, padding="same", activation='relu')(x)
x = layers.Conv1D(filters=CNN_2_FILTERS, kernel_size=CNN_2_SIZE, padding="same", activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
#x = layers.Dense(32, activation='relu')(x)
#x = layers.Flatten()(x)
x = layers.Dense(3, activation='sigmoid')(x)
model_cnn = tfkr.Model(tweet_input, x)
model_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 50)                0         
 ctorization)                                                    
                                                                 
 embedding_2 (Embedding)     (None, 50, 2)             4000      
                                                                 
 conv1d (Conv1D)             (None, 50, 24)            264       
                                                                 
 conv1d_1 (Conv1D)           (None, 50, 12)            1452      
                                                                 
 global_max_pooling1d (Glob  (None, 12)                0         
 alMaxPooling1D)                                           

In [17]:
with tf.device('device:GPU:0'):
    history_cnn = model_cnn.fit(X_train_ds, y_train, epochs=20, validation_split=0.2, batch_size=BATCH_SIZE, shuffle=True)

Epoch 1/20
512/512 [==============================] - 13s 21ms/step - loss: 0.7380 - accuracy: 0.6718 - val_loss: 0.4394 - val_accuracy: 0.8645
Epoch 2/20
512/512 [==============================] - 4s 8ms/step - loss: 0.3913 - accuracy: 0.8783 - val_loss: 0.3566 - val_accuracy: 0.8920
Epoch 3/20
512/512 [==============================] - 4s 7ms/step - loss: 0.3395 - accuracy: 0.8973 - val_loss: 0.3353 - val_accuracy: 0.8998
Epoch 4/20
512/512 [==============================] - 4s 8ms/step - loss: 0.3291 - accuracy: 0.9004 - val_loss: 0.3313 - val_accuracy: 0.9006
Epoch 5/20
512/512 [==============================] - 3s 7ms/step - loss: 0.3245 - accuracy: 0.9014 - val_loss: 0.3285 - val_accuracy: 0.9016
Epoch 6/20
512/512 [==============================] - 3s 6ms/step - loss: 0.3211 - accuracy: 0.9019 - val_loss: 0.3297 - val_accuracy: 0.9016
Epoch 7/20
512/512 [==============================] - 4s 7ms/step - loss: 0.3191 - accuracy: 0.9025 - val_loss: 0.3267 - val_accuracy: 0.9018
Epoc

In [30]:
y_pred = model_cnn.predict(X_test_ds)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test.to_numpy(), axis=1)
# prompt: calculate the accuracy for y_pred
accuracy = np.mean(y_pred == y_true)
print(f"Accuracy: {accuracy}")

2560/2560 [==============================] - 5s 2ms/step
Accuracy: 0.9042373916229087


## CNN-RNN Hybrid Model ##

In [27]:
EMBED_OUT_DIM = 2
EMBED_IN_DIM = VOCAB_SIZE
CNN_1_SIZE = 5
CNN_1_FILTERS = 16
RNN_HIDDEN_DIM = 12


tweet_input = tfkr.Input(shape=(1,), dtype=tf.string)
x = text_vectorize_layer(tweet_input)
x = layers.Embedding(input_dim=EMBED_IN_DIM, output_dim=EMBED_OUT_DIM, mask_zero=False)(x)
x = layers.Conv1D(filters=CNN_1_FILTERS, kernel_size=CNN_1_SIZE, padding="same", activation='relu')(x)
x = layers.LSTM(units=RNN_HIDDEN_DIM)(x)
#x = layers.GlobalMaxPooling1D()(x)
#x = layers.Dense(32, activation='relu')(x)
#x = layers.Flatten()(x)
x = layers.Dense(3, activation='sigmoid')(x)
model_hyb = tfkr.Model(tweet_input, x)
model_hyb.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_hyb.summary()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 50)                0         
 ctorization)                                                    
                                                                 
 embedding_10 (Embedding)    (None, 50, 2)             4000      
                                                                 
 conv1d_8 (Conv1D)           (None, 50, 16)            176       
                                                                 
 lstm_4 (LSTM)               (None, 12)                1392      
                                                                 
 dense_12 (Dense)            (None, 3)                 39        
                                                          

In [28]:
with tf.device('device:GPU:0'):
    history_hyb = model_hyb.fit(X_train_ds, y_train, epochs=20, validation_split=0.2, batch_size=BATCH_SIZE, shuffle=True)

Epoch 1/20
512/512 [==============================] - 16s 26ms/step - loss: 0.8875 - accuracy: 0.5415 - val_loss: 0.6288 - val_accuracy: 0.6925
Epoch 2/20
512/512 [==============================] - 6s 11ms/step - loss: 0.5182 - accuracy: 0.7978 - val_loss: 0.4331 - val_accuracy: 0.8655
Epoch 3/20
512/512 [==============================] - 5s 9ms/step - loss: 0.4180 - accuracy: 0.8686 - val_loss: 0.3943 - val_accuracy: 0.8759
Epoch 4/20
512/512 [==============================] - 5s 10ms/step - loss: 0.3760 - accuracy: 0.8832 - val_loss: 0.3642 - val_accuracy: 0.8877
Epoch 5/20
512/512 [==============================] - 4s 8ms/step - loss: 0.3564 - accuracy: 0.8905 - val_loss: 0.3559 - val_accuracy: 0.8927
Epoch 6/20
512/512 [==============================] - 5s 9ms/step - loss: 0.3436 - accuracy: 0.8964 - val_loss: 0.3519 - val_accuracy: 0.8946
Epoch 7/20
512/512 [==============================] - 4s 9ms/step - loss: 0.3360 - accuracy: 0.9000 - val_loss: 0.3376 - val_accuracy: 0.8999
Ep

In [29]:
y_pred = model_hyb.predict(X_test_ds,batch_size=256)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test.to_numpy(), axis=1)
# prompt: calculate the accuracy for y_pred
accuracy = np.mean(y_pred == y_true)
print(f"Accuracy: {accuracy}")

320/320 [==============================] - 2s 3ms/step
Accuracy: 0.8998656734644035
